In [37]:
# Iman Wahle and Jenna Kahn 
# August 2020
# Adapted from CFL_ElNino_Demonstration.ipynb by Krzysztof Chalupka
# This notebook walks through aggregating microlevel climate data to
# macrolevel features as causal hypotheses for future testing. The neural
# network used for density learning has been adapted from the original
# implementation to use tensorflow. 

In [2]:
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors

import core_ml_tf as cmt # new tf backend
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import cluster 

In [3]:
imshape = (55, 9)# This is the grid shape of our images, stored here for plotting reference.
#TODO: for other data, this shouldn't be hardcoded, right?

The Dataset
-----------

The following code loads our data into the correct format:
* X -- numpy array of size (n_datapoints, n_input_dim). Each row corresponds to one input value.
* Y -- numpy array of size (n_datapoints, n_output_dim). Each row is one output value.

In our case, each row of X is a (flattened) map of Pacific zonal wind strength, and each row of Y a (flattened) map of Pacific water temperature over the same region. In our case, n_input_dim == n_output_dim, but this need not be the case at all.

In [7]:
# Load the data. 

######### OLD METHOD TO LOAD DATA (from pkl)
## PLUG YOUR OWN DATA HERE. 'coords' is only needed to display climate maps. Your data
## can only contain X and Y arrays.
# uncomment to run 
X, Y, coords = joblib.load('elnino_data.pkl')


######### NEW METHOD TO LOAD DATA (from csv)
# # Note: make sure that you have the proper .csv files (x.csv, y.csv, and coords.csv) already 
# #if you don't, you can generate them by running writeElNinoFromPklToCSV.py
# X = np.genfromtxt("x.csv", delimiter=",")
# Y = np.genfromtxt("y.csv", delimiter=",")
#TODO: doesn't work yet (causes problems in training module)
# # # TODO: haven't dealt w coords yet    


# Create a randomized, normalized training and validation set.
x_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(Y)
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X, Y, shuffle=True, train_size=0.85)
X_tr = x_scaler.transform(X_tr).astype('float32')
Y_tr = y_scaler.transform(Y_tr).astype('float32')
X_ts = x_scaler.transform(X_ts).astype('float32')
Y_ts = y_scaler.transform(Y_ts).astype('float32')

/home/jkahn1618/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The file 'elnino_data.pkl' has been generated with a joblib version less than 0.10. Please regenerate this pickle file.
  import sys


In [8]:
print('X shape: {}'.format(X.shape))
print('Y.shape: {}'.format(Y.shape))

X shape: (13140, 495)
Y.shape: (13140, 495)


Learning P(Y | X)
--------------------
The first step of Causal Feature Learning (CFL) is to cluster x's according to P(Y | x) conditional densities. In this demonstration, for simplicity, we will approximate P(Y | x) with its expected value E[P(Y | x)]. This means we assume that if two distributions have equal means, they themselves are equal. It is possible to efficiently relax this assumption by using Mixture Density Networks (Bishop 1995) to approximate all moments of a distribution.

Learning E[P(Y | x)] amounts to regressing y on x. We do this using a neural network and the Lasagne package.

In [ ]:
cmt.train_network(X_tr, Y_tr, X_ts, Y_ts, 
                  save_fname='checkpoints/epoch_{}', 
                  n_epochs=3001,
                  lr=1e-3, verbose=True)

In [10]:
model = cmt.get_model(X.shape[1], Y.shape[1])
model.load_weights('checkpoints/epoch_2400')

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/epoch_2400

Finding the Observational Partition of X
-----------------------------------------
Finding the observational partition of X amounts to clustering the X data according to P(Y | X). That is, we put two x's in the same bucket if the neural net we trained maps them to the same values.

In [11]:
x_lbls = cluster.kMeansX(X, model)

Finding the Observational Partition of Y
---------------------------------------------
To find the partition of Y, we want to put together all y1 and y2 if P(y1 | x) == P(y2 | x) for each x. This procedure is a little bit more interesting than clustering the x's, and is described in our UAI 2016 paper. We describe it briefly here.

We've already clustered x's in a way that that guarantees that if x1 and x2 belong to the same x_lbls class, then P(y | x1) == P(y | x2). Thus, the requirement P(y1 | x) == P(y2 | x) can be subsituted by P(y1 | x_lbls==O) == P(y2 | x_lbls==O) for any observational X-class O. Since x_lbls is discrete, we should have plenty of data per each x_lbls class. We will approximate P(y | x_lbls==O) using the distance of y to the closest (except for itself) Y-point whose corresponding x belongs to x_lbls==O.

In [12]:
y_lbls = cluster.kMeansY(Y, x_lbls)

NameError: name 'x_lbls' is not defined

Understanding the Results
--------------------------
Visualizing the observational partition is data-specific. In our case, since both X and Y are images, we can visualize the means of each observational cluster to gain some insight into what it contains. 

But, one of the virtues of the method is that it is interpretation-agnostic. The observational partition can be used as a causal hypothesis to drive experimentation. This can be done whether the inputs and outputs are easily interpretable or not at all.

In [ ]:
fig = plt.figure(figsize=(15,10), facecolor='white')
X_raw = x_scaler.inverse_transform(np.vstack([X_tr, X_ts]))
Y_raw = y_scaler.inverse_transform(np.vstack([Y_tr, Y_ts]))

levels = np.linspace(-0.5,0.5,30)
for x_cluster_id in range(4):
    ax = plt.subplot2grid((4,2), (x_cluster_id, 0))  
    # Plot the cluster's mean difference from all frames' mean.
    cluster_mean = (X_raw[x_lbls==x_cluster_id].mean(axis=0)-X_raw.mean(axis=0)).reshape(imshape).T
    im=ax.contourf(coords['x'].ravel(), coords['y'].ravel(), cluster_mean, levels=levels, cmap='BrBG_r')
    ax.set_xticks([]); ax.set_yticks([])

levels=np.linspace(-1,1.5,30)
for y_cluster_id in range(4):
    ax = plt.subplot2grid((4,2), (y_cluster_id, 1))  
    # Plot the cluster's mean difference from all frames' mean.
    cluster_mean = (Y_raw[y_lbls==y_cluster_id].mean(axis=0)-Y_raw.mean(axis=0)).reshape(imshape).T

    im=ax.contourf(coords['x'].ravel(), coords['y'].ravel(), cluster_mean, levels=levels, cmap='coolwarm')
    ax.set_xticks([]); ax.set_yticks([])

# Compute and print P(y_lbl | x_lbl)
P_CE = np.array([np.bincount(y_lbls.astype(int)[x_lbls==x_lbl], 
    minlength=y_lbls.max()+1).astype(float) for x_lbl in np.sort(np.unique(x_lbls))])
P_CE = P_CE/P_CE.sum()
P_E_given_C = P_CE/P_CE.sum(axis=1, keepdims=True)

print('P(TempCluster | WindCluster):')
print(P_E_given_C)